# Convlution Autoencoder
- Prediction
- Save image id and predicted crop size for each test data
- Crop test images


# U-Net

- Apply any preprocessing for U-Net
- Amend custom dataset for the second model
